### Imports

In [1]:
import pandas as pd
import numpy as np
import re
import calendar
from collections import Counter
from collections import OrderedDict

# Data Cleaning

#### Creating Dataframe

In [2]:
df = pd.read_csv(r'Raw LinkedIn Dataset.csv')

#### Naming columns

In [3]:
col = ['Role', 'Company', 'Location', 'Time posted', 'No of applicants', 'Level', 'Job type', 'Job function', 'Industry']
df.columns = col

#### Cleaning columns individually

In [4]:
# Cleaning "Company" column  
df['Company'] = df['Company'].apply(lambda x: x.replace(' ', '', 16).split('\n')[1])

# "Location" column
df['Location'] = df['Location'].apply(lambda x: x.replace(' ', '', 14).split('\n')[1])

# "Time posted" column
df['Time posted'].fillna(0, inplace=True)
df['Time posted'] = df['Time posted'].apply(lambda x: x.split('\n')[-3].replace(' ', '', 4) if (type(x)!=int) else 'Not available')

# "Level" column
df['Level'] = df['Level'].apply(lambda x: re.findall('[\w-]+[ ]?\w+',x)[0])

# "Job type" column
df['Job type'] = df['Job type'].apply(lambda x: x.replace(' ', '', 10).split('\n')[1] if type(x)==str else ('Not mentioned'))

# "Job Function" column
df['Job function'] = df['Job function'].apply(lambda x: x.replace(' ', '', 12).split('\n')[1] if (type(x)!=float) else 'Not mentioned')

# "Industry" column
df['Industry'] = df['Industry'].apply(lambda x: x.replace(' ', '', 10).split('\n')[1] if (type(x)!=float) else 'Not mentioned')

#### Dropping unwanted column

In [5]:
del df['No of applicants']
# since over 70% of data in No.applicant are 'nan', it was dropped

# Data Wrangling

#### Extracting Name of the city from "Location" column

In [6]:
city_dict = {'Mumbai Metropolitan Region': 'Mumbai',
            'Pune/Pimpri-Chinchwad Area': 'Pune',
            'India': 'Not mentioned',
            'Chandigarh': 'Chandigarh',
            'Punjab': 'Not mentioned',
            'Karnataka': 'Not mentioned',
            'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli',
            'Goa': 'Goa'}


def city(x):
    c = x.split(',')
    c1 = c[0]
    if len(c) > 2:
        return c1
    elif c1 in city_dict.keys():
        return city_dict[c1]
    elif 'Greater' in c1:
        return c1.split(' ')[1]


df['City'] = df['Location'].apply(city)

#### Extracting Name of the state from "Location" column

In [7]:
state_dict = {'Mumbai Metropolitan Region': 'Maharashtra',
            'Greater Nashik Area': 'Maharashtra',
            'Pune/Pimpri-Chinchwad Area': 'Maharashtra',
            'Greater Madurai Area': 'Tamil Nadu',
            'Greater Delhi Area': 'Delhi',
            'Greater Hyderabad Area': 'Telangana',
            'India': 'Not mentioned'}


def state(x):
    s = x.split(',')
    if len(s) > 2:
        return s[1].replace(' ', '', 1)
    elif len(s) == 2:
        return s[0]
    elif x in state_dict.keys():
        return (state_dict[x])


df['State'] = df['Location'].apply(state)

#### Standardizing city names

In [8]:
def grp(x):
    if 'Bangalore Urban' in x or 'Bengaluru North' in x or 'Bengaluru Rural' in x:
        return 'Bengaluru'
    elif 'Open Area' in x or 'Okhla' in x or 'Sarita Vihar' in x:
        return 'Delhi'
    elif 'Trivandrum' in x:
        return 'Thiruvananthapuram'
    elif 'Serilingampally' in x:
        return 'Hyderabad'
    elif 'Kochi' in x:
        return 'Ernakulam'
    else:
        return x


df['Grouped City'] = df['City'].apply(grp)

#### Extracting days from "Time posted" column

In [9]:
def days(x):
    if x != 'Not available':
        if 'week' in x:
            return int(x.split(' ')[0])*7
        elif 'month' in x:
            return int(x.split(' ')[0])*31
        elif 'day' in x:
            return int(x.split(' ')[0])
    else:
        return 0


df['No. of days ago'] = df['Time posted'].apply(days)

#### Extracting month from "No. of days ago" column

In [10]:
def month(x):
    if x != 0:
        td = pd.Timedelta(days=x)
        m = ts-td
        return (calendar.month_name[m.month])
    else:
        return 'Not mentioned'


ts = pd.Timestamp('2021/11/29') # Date on which data was scrapped from linkedin
df['Month posted'] = df['No. of days ago'].apply(month)

# Analysis

#### Counting number of times an individual term mentioned in job function

In [11]:
val = df['Job function']
terms = []
for i in val:
    l = re.split(',| and',i)
    for ii in range(len(l)):
        if ii > 0:
            terms.append(l[ii].replace(' ', '', 1))
        else:
            terms.append(l[ii])

terms_counter = Counter(terms)
terms_counter = dict(terms_counter)
del terms_counter['']
del terms_counter['Not mentioned']

# Creating a dataframe for obtained result for export
job_functions = pd.DataFrame({'Functions': terms_counter.keys(), 'Total mentions': terms_counter.values()})

#### Counting number of time a individual term mentioned in Industry

In [12]:
val = df['Industry']
terms = []
for i in val:
    ll = i.replace(', and', ',')
    l = re.split(',', ll)
    for ii in range(len(l)):
        if ii > 0:
            terms.append(l[ii].replace(' ', '', 1))
        else:
            terms.append(l[ii])

terms_counter = Counter(terms)
terms_dict = dict(terms_counter)
del terms_dict['Not mentioned']
terms_dict

# Creating a dataframe for obtained result for export
industry = pd.DataFrame({'Industry': terms_dict.keys(), 'Total mentions': terms_dict.values()})

#### Modifying columns

In [13]:
del df['Location']
del df['Time posted']
del df['City']
del df['Role']

# Renaming column
df.rename(columns={'Grouped City': 'City'}, inplace=True)

# Rearranging columns
column = ['Company', 'City', 'State', 'Month posted', 'No. of days ago', 'Level', 'Job type','Job function', 'Industry']
df = df[column]

# Exporting data